В качестве модели для решения данной задачи была взята модель NaiveBayes в предположении, что ~80% данных, поступающих в модель, являются "человеческим" текстом.

В качестве строк кода на языке python были взяты 2 репозитория: https://github.com/3b1b/manim (тренировочный) и https://github.com/openai/gym (тестовый). Брались все файлы с расширением py и все строчки кода оттуда (кроме комментариев и строчных констант).

В качестве строк на "человеческом" языке были взяты файлы с расширениями txt и md из этих репозиториев.
Однако, данных оттуда не хватило даже для того, чтобы модель отмечала строчки "yield from range(number + 1, number + on_each_side + 1)" и "If the page range is larger than a given size, the whole range is not" как строчки кода и "человеческого" языка соответственно.

По этой причине были взяты также файлы с расширением txt (с сайта https://www.homeenglish.ru/Books.htm), содержащие некоторые литературные произведения, написанные на английском языке.

In [1]:
import os
import re
import numpy as np
from sklearn.naive_bayes import MultinomialNB as NaiveBayes
from sklearn.feature_extraction.text import CountVectorizer as BoW
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# путь к файлу -> список из строк этого файла
def read(path, code=True):
    with open(path, 'r') as f:
        try:
            text = f.read()
        except Exception:
            print(path)
            text = ''
    if code:  # удаляем комментарии из нескольких строк и str из кода
        text = re.sub(r"'''(.|\n)*'''", '', text)
        text = re.sub(r'"""(.|\n)*"""', '', text)
        text = re.sub(r"'.*'", '', text)
        text = re.sub(r'".*"', '', text)
    else:  # удаляем кавычки из "человеческих" текстов
        text = re.sub(r'"(.*)"', r'\1', text)
    new_text = []
    for line in text.split('\n'):
        line = line.strip()
        if code:  # удаляем однострочные комментарии из кода
            pos = re.search('#', line)
            if pos:
                line = line[:pos.span()[0]]
        if line:  # пустые строки не нужны
            new_text.append(line)
    return new_text

In [3]:
# обходит всё внутри path, собирает строки кода и строки "человеческого" текста из файлов
def get_corpuses(path):
    code_corpus, text_corpus = [], []
    for root, dirs, files in os.walk(path):
        for file in files:
            fullname = os.path.join(root, file)
            if re.search(r'\.py', file):
                code_corpus.extend(read(fullname))
            if re.search(r'\.(txt|md)', file):
                text_corpus.extend(read(fullname, code=False))
    return code_corpus, text_corpus

In [4]:
# получаем текстовые данные
train_code_corpus, train_text_corpus = get_corpuses(r'train_data')

In [5]:
# переводим текстовые данные в вектора
vectorizer = BoW()
X_train = vectorizer.fit_transform(train_code_corpus + train_text_corpus)
y_train = np.array([1] * len(train_code_corpus) + [0] * len(train_text_corpus))

In [6]:
# обучаем NaiveBayes на полученных векторах
nb = NaiveBayes(alpha=0.01, class_prior=(0.8, 0.2))
nb.fit(X_train, y_train)

MultinomialNB(alpha=0.01, class_prior=(0.8, 0.2))

In [7]:
# Проверяем пример из файла с заданием
test_code = "yield from range(number + 1, number + on_each_side + 1)"
test_text =  "If the page range is larger than a given size, the whole range is not"
nb.predict(vectorizer.transform([test_code, test_text]))

array([1, 0])

На данном примере всё работает так, как ожидалось.

Теперь протестируем модель на большем количестве примеров.

Количество примеров взято так, чтобы было ~80% "человеческого" текста в тренировочной выборке.

In [8]:
# Данные для тестирования (в текстовом виде)
test_code_corpus, test_text_corpus = get_corpuses(r'test_data')

In [9]:
# соотношения количества строк текстовых данных к количеству строк в коде
print(len(train_text_corpus) / len(train_code_corpus))
print(len(test_text_corpus) / len(test_code_corpus))

5.516010978956999
4.051377586037847


In [10]:
# Проверка качества модели
X_test = vectorizer.transform(test_code_corpus + test_text_corpus)
y_test = np.array([1] * len(test_code_corpus) + [0] * len(test_text_corpus))
y_pred = nb.predict(X_test)
p, r, f, s = precision_recall_fscore_support(y_test, y_pred)
print(f'Precision: {p[1]}')
print(f'Recall: {r[1]}')

Precision: 0.9936578404946884
Recall: 0.6144720070595157


Результаты тестирования говорят нам о том, что было выявлено всего около 60% строчек кода.

При этом, если модель определила поданную ей на вход строку как строчку кода, то она с очень большой вероятностью (близкой к 1) определила верно.

Модель не очень пригодна к полному использованию на практике, но благодаря её использованию можно перепроверять лишь только те строки, которые были классифицированы как "человеческий" текст, что сильно упрощает работу.

Можно также поэкспериментировать с датасетами, на которых обучаем и тестируем модель (брать в качестве "человеческого текста" больше комментариев к коду, а не литературные произведения), и с различными векторизациями строчек.